In [ ]:
!pip -q install llama-index-llms-huggingface==0.1.5 \
                llama-index-embeddings-huggingface==0.2.0 \
                loralib==0.1.2 \
                sentencepiece==0.1.99 \
                bitsandbytes==0.43.0 \
                accelerate==0.28.0 \
                llama-index==0.10.33 \
                newspaper3k==0.2.8 \
                langchain==0.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 51.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 23.2 M

In [ ]:
!pip install flash-attn --no-build-isolation -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 30.1 MB/s eta 0:00:00


In [ ]:
import os
import requests
import torch
import transformers


from huggingface_hub import HfApi, HfFolder

from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          PreTrainedTokenizer,
                          PreTrainedModel,
                          BitsAndBytesConfig,
                          pipeline
                        )

from llama_index.core import (SummaryIndex,
                              VectorStoreIndex,
                              SimpleDirectoryReader,
                              StorageContext,
                              load_index_from_storage,
                              Settings,
                              PromptTemplate
)

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.llms import ChatMessage

from langchain.document_loaders import NewsURLLoader

import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [ ]:
# Hugging Face access token
hf_token = "your_access_token"

# HfFolder to save the token for subsequent API calls
HfFolder.save_token(hf_token)


In [ ]:
# Infos about chat template for llama 3: https://github.com/meta-llama/llama-recipes
system_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
                    You are a helpful, respectful, and honest assistant.
                    <|eot_id|><|start_header_id|>user<|end_header_id|>
                """

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("{query_str}<|eot_id|><|start_header_id|>assistant<|end_header_id|>")

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)

stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

# BitsAndBytes configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_in_8bit=False, # You can optionally load it in 8bit
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="fp4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

llm = HuggingFaceLLM(
    model_name=model_id,
    max_new_tokens=512,
    model_kwargs={
        "token": hf_token,
        "quantization_config": bnb_config
    },
    generate_kwargs={
        "do_sample": True,
        "temperature": 0.6,
        "top_p": 0.9,
    },
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=model_id,
    tokenizer_kwargs={"token": hf_token},
    stopping_ids=stopping_ids,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
Settings.llm = llm

In [ ]:
# Loading the news
news = NewsURLLoader(urls=[
    'https://finance.yahoo.com/news/nvidias-gtc-was-ceo-jensen-huangs-big-moment-173007915.html',
    'https://finance.yahoo.com/news/stock-market-today-sp-500-hits-fresh-record-as-all-eyes-turn-to-fed-decision-200457552.html',
    'https://finance.yahoo.com/video/doj-accuses-apple-iphone-monopoly-151205746.html',
    'https://finance.yahoo.com/m/4205eaa9-f620-3a0b-a81a-0e82c7c9fd0b/magnificent-seven-stocks-to.html'
]).load()


# Storing in a DF
news_content = [c.page_content for c in news]
news_headline = [c.metadata['title'] for c in news]

news_df = pd.DataFrame({'News_Headline': news_headline})
news_df['News_Text'] =  news_content

In [ ]:
news_df

,News_Headline,News_Text
0,Nvidia's GTC was CEO Jensen Huang's big moment,Nvidia CEO Jensen Huang might as well have pit...
1,Stock market today: S&P 500 hits fresh record ...,Wall Street closed the trading session on a hi...
2,DOJ accuses Apple of iPhone monopoly in landma...,The US Justice Department and 16 state and dis...
3,"Magnificent Seven Stocks: Nvidia Stock, Tesla ...",In an era in which the power to shape AI may u...


In [ ]:
def extract_sentiment(chat_response):
    # Access the content directly from the chat_response object
    content = chat_response.message.content

    # The sentiment word is located after 'assistant\n\n' in the content string
    # Split the string at 'assistant\n\n' and take the second part
    parts = content.split('assistant\n\n')
    if len(parts) > 1:
        # The sentiment word is expected to be the first word after the split
        sentiment = parts[1].split()[0]  # Split by whitespace and take the first word
        return sentiment
    else:
        return None

In [ ]:
# Function to get sentiment
def get_sentiment(text):
    # Formatted instruction string with dynamic text input
    instruction = f"""Classify the text into neutral, negative, or positive. Answer only with the word.
                    Text: {text}
                    """

    messages = [
        ChatMessage(role="system", content="You are an expert in sentiment classification."),
        ChatMessage(role="user", content=instruction),
    ]

    # Make the API call and store the response
    response = llm.chat(messages)
    sentiment = extract_sentiment(response)

    return sentiment

In [ ]:
# Apply the function over each row in the DataFrame and create a new column for sentiment
news_df['Sentiment'] = news_df['News_Headline'].apply(get_sentiment)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
news_df

,News_Headline,News_Text,Sentiment
0,Nvidia's GTC was CEO Jensen Huang's big moment,Nvidia CEO Jensen Huang might as well have pit...,Positive
1,Stock market today: S&P 500 hits fresh record ...,Wall Street closed the trading session on a hi...,Positive
2,DOJ accuses Apple of iPhone monopoly in landma...,The US Justice Department and 16 state and dis...,Negative
3,"Magnificent Seven Stocks: Nvidia Stock, Tesla ...",In an era in which the power to shape AI may u...,Negative


In [ ]:
sent_headline = news_df['News_Headline'].iloc[0]

In [ ]:
instruction = f"""Classify the text into neutral, negative, or positive. Provide
                      a reasoning why you classified it as neutral, negative, or positive.
                      For instance if you find words like 'big moment', this is likely positive.
                      Text: {sent_headline}
                      Reason: """

messages = [
        ChatMessage(role="system", content="You are an expert in sentiment classification."),
        ChatMessage(role="user", content=instruction),
    ]

response = llm.chat(messages)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
response

ChatResponse(message=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='assistant\n\nI would classify the text as positive.\n\nReasoning: The phrase "big moment" is a positive phrase, implying that the event or occasion was significant and notable. Additionally, the mention of CEO Jensen Huang\'s name suggests that the text is highlighting his achievement or success, which is often associated with positive connotations. The fact that the text specifically mentions Nvidia\'s GTC (GPU Technology Conference) further reinforces the positive tone, as it implies a sense of accomplishment or achievement in the tech industry.', additional_kwargs={}), raw={'model_output': tensor([[128000, 128000, 128006,   9125, 128007,    271,   2675,    527,    459,
           6335,    304,  27065,  24790,     13, 128009, 128006,    882, 128007,
            271,   1999,   1463,    279,   1495,   1139,  21277,     11,   8389,
             11,    477,   6928,     13,  40665,    198,   2611,    264,